In [7]:
#Sunday 4:00pm PST is time to run
import csv
import math
from statistics import median
from DBconnect import myDB
#using Binance
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import pandas as pd
import numpy as np

f = open('binance.key', 'r')
client = Client(f.readline().strip(), f.readline().strip(), tld='us')

In [18]:
mydb = myDB()
mydb.load_db('db.key')
mydb.dbconnect()

In [4]:
#Getting All symbols from Binance US
#Get 20 symbols per IP to not get blocked
def getBiSymbols():
    raw_symbols = client.get_all_tickers()
    bi_symbols = []

    for s in range(len(raw_symbols)):
        bi_symbols.append(raw_symbols[s]['symbol'])

    for i in range(3):
        for w in bi_symbols:
            if w.endswith('USD'):
                pass
            else:
                bi_symbols.remove(w)
    for i in range(3):
        for w in bi_symbols:
            if w.endswith('BUSD'):
                bi_symbols.remove(w)
            else:
                pass
    return bi_symbols

def getSymbols():
    mydb = myDB()
    mydb.load_db('db.key')
    mydb.dbconnect()
    symbols = []
    for elem in pd.read_sql_table('hltable', mydb.engine).columns:
        if elem == 'time':
            continue
        elif elem == 'index':
            continue
        elif elem == 'level_0':
            continue
        elif elem == 'level_1':
            continue
        symbols.append(elem.split('-')[0])
    return list(set(symbols))

def addNewWeekData():
    mydb = myDB()
    mydb.load_db('db.key')
    mydb.dbconnect()
    symbols = getSymbols()
    updatetable = getHLTable(symbols[:], "1 week ago UTC")
    hltable = pd.read_sql_table('hltable', mydb.engine)
    df = pd.concat([updatetable, hltable]).reset_index(drop = True)
    df.to_sql('hltable', con=mydb.engine, if_exists='replace')

In [5]:
hltable=pd.DataFrame()

In [8]:
bi_symbols = getBiSymbols()
len(bi_symbols)

66

In [13]:
#Get 20 symbols per IP to not get blocked
for bs in range(40,66):
    klines = client.get_historical_klines(bi_symbols[bs], Client.KLINE_INTERVAL_1WEEK, "1 Jan, 2020")
    df = pd.DataFrame(klines,columns = ['OpenTime','Open','High','Low','Close','Volume','CloseTime','QAV','Trades','TBAV','TQAV','ignore'])
    df = df.iloc[::-1]
    if bs == 0:
        hltable['time'] = df['OpenTime']
    hltable[bi_symbols[bs]+"-Close"] = df['Close']
    hltable[bi_symbols[bs]+"-High"] = df['High']
    hltable[bi_symbols[bs]+"-Low"] = df['Low']


hltable.dropna(axis='columns', inplace=True)

In [16]:
hltable.reset_index(drop = True, inplace = True)

In [19]:
hltable.to_sql('hltable', con=mydb.engine, if_exists='replace')

In [93]:
def getHLCompareTable(hltable):
    hlCompareTable=pd.DataFrame()
    hlCompareTable2=pd.DataFrame()

    #hlCompare Table saves Area, hlcompareTable2 saves High Price 
    tlen = hltable.shape[0]
    for col in hltable.columns:
        currency = col.split('-')[0]
        if col == 'time':
            continue
        elif col == 'index':
            continue
        elif col == 'level_0':
            continue
        elif col == 'level_1':
            continue
        elif currency in hlCompareTable.columns and currency in hlCompareTable2.columns:
            continue
        else:
            valueList = np.zeros(shape=(tlen-1,1))
            secondValueList = np.zeros(shape=(tlen-1,1))
            for i in range(tlen-1):
                try:
                    c = float(hltable[currency+'-Close'][i+1])
                except:
                    print(i)
                    print(currency)
                h = float(hltable[currency+'-High'][i])
                l = float(hltable[currency+'-Low'][i])

                if pd.isna(c) and pd.isna(h) and pd.isna(l):    
                    break
                else:
                    valueList[i][0] =((h-l)/c*100)
                    secondValueList[i][0] =((h-c)/c*100)
            hlCompareTable[currency] = pd.DataFrame(valueList, columns = [currency])[currency]
            hlCompareTable2[currency] = pd.DataFrame(secondValueList, columns = [currency])[currency]
    return hlCompareTable, hlCompareTable2


In [30]:
# Get table of runners: calculated base price and runner price
# CalcTable saves area difference, CalcTable2 saves high price difference.
def getHLCalcTable(hlCompareTable, hlCompareTable2):
    tlen = hlCompareTable.shape[0]
    hlCalcTable=pd.DataFrame()
    hlCalcTable2=pd.DataFrame()

    for col in hlCompareTable.columns:
        if col == 'BTCUSD':
            pass
        elif col in hlCalcTable.columns and col in hlCalcTable2.columns:
            pass
        else:
            calcList = np.zeros(shape=(tlen-1,1))
            secondCalcList = np.zeros(shape=(tlen-1,1))
            for i in range(tlen-1):
                baseArea = float(hlCompareTable['BTCUSD'][i+1])
                runnerArea = float(hlCompareTable[col][i])

                baseHigh = float(hlCompareTable2['BTCUSD'][i+1])
                runnerHigh = float(hlCompareTable2[col][i])

                calcList[i][0] = baseArea - runnerArea
                secondCalcList[i][0] = baseHigh - runnerHigh

            hlCalcTable[col] = pd.DataFrame(calcList, columns = [col])[col]
            hlCalcTable2[col] = pd.DataFrame(secondCalcList, columns = [col])[col]
    return hlCalcTable, hlCalcTable2

In [31]:
# Get a table of similarly moved runners
def getSimilarTable(hlCalcTable, hlCalcTable2, hlCompareTable, hlCompareTable2):
    tlen = hlCalcTable.shape[0]
    similarTable=pd.DataFrame()
    signalCol = []
    signalArea = []
    signalHigh = []
    resultHigh = []
    signalIndex = []
    for col in hlCalcTable.columns:

        for i in range(tlen):
            if abs(hlCalcTable[col][i]) < 1:
                if hlCalcTable2[col][i] < 1:
                        signalArea.append(round(hlCompareTable['BTCUSD'][i+1],2))
                        signalHigh.append(round(hlCompareTable2['BTCUSD'][i+1],2))
                        resultHigh.append(round(hlCompareTable2[col][i],2))
                        signalIndex.append(i+1)
                        signalCol.append(col)
    try:
        similarTable['Signal Col'] = pd.Series(signalCol)
        similarTable['Signal Area'] = pd.Series(signalArea)
        similarTable['Signal High'] = pd.Series(signalHigh)
        similarTable['Result High'] = pd.Series(resultHigh)
        similarTable['Signal Index'] = pd.Series(signalIndex)
    except:
            print("An exception occurred")
    similarTable = similarTable.loc[similarTable['Result High'] > 1]
    return similarTable

In [32]:
#Getting Max signaled currency
def getPivotTable(similarTable):
    pivotTable = similarTable.pivot_table(columns=['Signal Col'], aggfunc='size')
    pivotTable.sort_values(ascending=False, inplace =True)
    return pivotTable

def getMaxList(pivotTable):
    maxSignal = pivotTable.max()

    maxSeries = pd.DataFrame(pivotTable.where(pivotTable.eq(maxSignal))).stack()
    maxList = []
    for i in range(maxSeries.size):
        maxList.append(maxSeries.index[i][0])
    return maxList

In [90]:
# Get most high currency among max signaled currencies
def getBuySignal(pivotTable, similarTable, hlCompareTable, hlCompareTable2):
    maxUnit = 'None'
    buySignal = False
    for m in pivotTable.index:
        signalIndexList = []
#         if pivotTable[m] < 5:
#             break
        indexList = similarTable.where(similarTable['Signal Col'] == m).dropna().index
        signalHighList = []
        signalAreaList = []
        for i in indexList:
            signalHighList.append(similarTable.where(similarTable['Signal Col'] == m).dropna()['Signal High'][i])
            signalAreaList.append(similarTable.where(similarTable['Signal Col'] == m).dropna()['Signal Area'][i])

        signalEvaluateTable = hlCompareTable[['BTCUSD', m]]
        signalEvaluateTable['BTCUSD-High'] = hlCompareTable2['BTCUSD']
        signalEvaluateTable[m+'-High'] = hlCompareTable2[m]

        for i in range(len(signalHighList)):
            sIndexList = signalEvaluateTable.loc[(signalEvaluateTable['BTCUSD'] > signalAreaList[i]-0.5) & (signalEvaluateTable['BTCUSD'] < signalAreaList[i]+0.5) & (signalEvaluateTable['BTCUSD-High'] < signalHighList[i]+0.5) & (signalEvaluateTable['BTCUSD-High'] < signalHighList[i]+0.5)].index
            for s in sIndexList:
                signalIndexList.append(s)

        #if signalIndexList contains 0 -> buy signal
        # print(set(signalIndexList))
        if signalIndexList.count(0) > 0:
            buySignal = True
            maxUnit = m
            break
        else:
            buySignal = False
    return maxUnit, buySignal, signalIndexList

In [34]:
# Result Median High is expected profit %
def getMed(signalIndexList):
        sigCount = 0
        sigResultList = []
        for idx in signalIndexList:
            try:
                sigResultList.append(hlCompareTable2[maxUnit][idx-1])
                sigCount+=1
            except:
                pass
        if sigResultList == []:
            med = 0
        else:
            med = median(sigResultList)
        return med

def updateDB(maxUnit, buySignal, signalIndexList, target):
    if buySignal:
        print('Signal Unit: ', maxUnit)        
        print('Result median High: ', round(target,2))
        print('Signal Count : ',len(sigResultList))

        mydb.runQuery(maxUnit, round(target,2),"True")

    else:
        mydb.runQuery('None', 0,"False")
        print('NO SIGNAL')

In [38]:
hltable.shape[0]

105

In [103]:
#need 2 y data for back testing
#need to import data using different IP to test with live data
result = 1
for i in range(1,hltable.shape[0]-52):
    testtable = pd.read_sql_table('hltable', mydb.engine)[i:i+52]
    testtable.reset_index(inplace = True, drop = True)
    hlCompareTable, hlCompareTable2 = getHLCompareTable(testtable)
    hlCalcTable, hlCalcTable2 = getHLCalcTable(hlCompareTable, hlCompareTable2)

    #pass 2 y data to back test
    similarTable = getSimilarTable(hlCalcTable, hlCalcTable2, hlCompareTable, hlCompareTable2)

    pivotTable = getPivotTable(similarTable)

    maxList = getMaxList(pivotTable)

    maxUnit, buySignal, signalIndexList = getBuySignal(pivotTable, similarTable, hlCompareTable, hlCompareTable2)

    #If buySignal = Ture then, check if next_series_hlCompareTable2[0] > target
    #If true, result *= target
    #If false, result *= next_series_hltable-Close[-2] - next_series_hltable-Close[-1]
    target = getMed(signalIndexList)
    print(buySignal)
    if buySignal == True:
        if (float(hltable[maxUnit+'-High'][i-1]) - float(hltable[maxUnit+'-Close'][i]))/float(hltable[maxUnit+'-Close'][i])*100 > target:
            result *= 1+target/100
        else:
            result *= 1+(float(hltable[maxUnit+'-Close'][i-1]) - float(hltable[maxUnit+'-Close'][i]))/100
    if buySignal == False:
        pass
    print(result)

<ipython-input-90-9eedb7eb5aba>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signalEvaluateTable['BTCUSD-High'] = hlCompareTable2['BTCUSD']
<ipython-input-90-9eedb7eb5aba>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signalEvaluateTable[m+'-High'] = hlCompareTable2[m]
<ipython-input-90-9eedb7eb5aba>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

<ipython-input-90-9eedb7eb5aba>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signalEvaluateTable['BTCUSD-High'] = hlCompareTable2['BTCUSD']
<ipython-input-90-9eedb7eb5aba>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signalEvaluateTable[m+'-High'] = hlCompareTable2[m]


False
1


<ipython-input-90-9eedb7eb5aba>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signalEvaluateTable['BTCUSD-High'] = hlCompareTable2['BTCUSD']
<ipython-input-90-9eedb7eb5aba>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signalEvaluateTable[m+'-High'] = hlCompareTable2[m]
<ipython-input-90-9eedb7eb5aba>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

<ipython-input-90-9eedb7eb5aba>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signalEvaluateTable[m+'-High'] = hlCompareTable2[m]
<ipython-input-90-9eedb7eb5aba>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signalEvaluateTable['BTCUSD-High'] = hlCompareTable2['BTCUSD']
<ipython-input-90-9eedb7eb5aba>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

...

<ipython-input-90-9eedb7eb5aba>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signalEvaluateTable['BTCUSD-High'] = hlCompareTable2['BTCUSD']
<ipython-input-90-9eedb7eb5aba>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signalEvaluateTable[m+'-High'] = hlCompareTable2[m]


False
1.9622639985087753


<ipython-input-90-9eedb7eb5aba>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signalEvaluateTable['BTCUSD-High'] = hlCompareTable2['BTCUSD']
<ipython-input-90-9eedb7eb5aba>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signalEvaluateTable[m+'-High'] = hlCompareTable2[m]
<ipython-input-90-9eedb7eb5aba>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

False
1.9622639985087753


<ipython-input-90-9eedb7eb5aba>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signalEvaluateTable['BTCUSD-High'] = hlCompareTable2['BTCUSD']
<ipython-input-90-9eedb7eb5aba>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signalEvaluateTable[m+'-High'] = hlCompareTable2[m]
<ipython-input-90-9eedb7eb5aba>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

In [104]:
result


1.9622639985087753

In [107]:
pd.read_sql_table('hltable', mydb.engine)

,index,time,BTCUSD-Close,BTCUSD-High,BTCUSD-Low,ETHUSD-Close,ETHUSD-High,ETHUSD-Low,BCHUSD-Close,BCHUSD-High,...,QTUMUSD-Low,NANOUSD-Close,NANOUSD-High,NANOUSD-Low,ICXUSD-Close,ICXUSD-High,ICXUSD-Low,ENJUSD-Close,ENJUSD-High,ENJUSD-Low
0,104,1641168000000,41836.1500,47568.9500,40500.0000,3122.1000,3893.4700,2999.0600,376.7700,447.2000,...,7.3880,3.1519,3.9500,2.9271,0.9924,1.2749,0.9549,2.2646,2.9493,2.1711
1,103,1640563200000,47302.8300,52148.5000,45665.9200,3829.8600,4129.1200,3579.6700,448.0100,475.9800,...,8.4770,3.5801,4.1165,3.3630,1.2629,1.9648,1.1816,2.8626,3.2494,2.5028
2,102,1639958400000,50792.3700,51853.6500,45566.2200,4063.3700,4160.4900,3755.1100,451.7500,461.1900,...,8.6370,3.8746,3.9465,3.1525,1.5417,1.6853,1.1355,3.1958,3.3436,2.1636
3,101,1639353600000,46681.4100,50214.1200,45471.5700,3925.6000,4146.0000,3645.1000,432.4300,459.2000,...,8.6240,3.3440,3.9457,3.1500,1.2018,1.2765,1.0615,2.3351,2.7630,2.1913
4,100,1638748800000,50090.6100,51992.0700,46716.1400,4133.4400,4491.3800,3833.6200,457.9500,502.6700,...,8.9910,3.9240,4.3053,3.5661,1.2402,1.3981,1.0669,2.6927,3.0102,2.3639
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,4,1580688000000,10174.7500,10174.7500,9080.2400,228.7300,231.0300,184.3900,450.0900,461.2900,...,2.1680,0.9719,1.0208,0.7953,0.4392,0.5986,0.3120,0.1326,0.1556,0.1058
101,3,1580083200000,9326.4500,9564.6900,8564.0200,188.7300,193.4400,165.2200,376.7600,397.4400,...,1.9370,0.8296,0.8612,0.6805,0.3189,0.7520,0.1751,0.1187,0.1221,0.0850
102,2,1579478400000,8593.6600,8795.0000,8222.0000,167.4000,171.0100,155.3200,346.6700,353.0000,...,1.8070,0.6830,0.7165,0.6387,0.1750,0.1795,0.1445,0.0876,0.0946,0.0739
103,1,1578873600000,8701.1200,9190.9200,8034.2300,166.7900,179.2400,142.2800,338.5500,404.4500,...,1.6840,0.6807,0.7656,0.6140,0.1447,0.1580,0.1196,0.0740,0.0878,0.0700
